In [1]:
import tweepy
import csv
import pandas as pd
import pickle
import re
import statistics as st
import datetime
import plotly.express
pd.options.plotting.backend = "plotly"

In [ ]:
#keys
consumer_key = "w0ywEBjife6WBfIVOgMffvlvE"
consumer_secret = "5T64JvO3J4ftnjyIQolE0J17w0dAXeEDFS9GsyVrzBHgLM5au9"
access_key = "1361669854269476868-fTDqiXUegqTDQT8tPU5UvMQAxuXGV6"
access_secret = "zBITqkEyK1A95epa2pn25RrxDs2QnWbs2Ys5am5Pyn0bi"

In [ ]:
#tweepy api
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

In [ ]:
#Data collection
tweets=[]
for status in tweepy.Cursor(api.user_timeline,id='DcpNorthDelhi',tweet_mode='extended').items():
    tweets.append(status)

In [ ]:
#storing tweets in pkl file
# open_file = open("DcpNorthdelhiTweets.pkl", "wb")
# pickle.dump(tweets, open_file)
# open_file.close()

In [ ]:
#reading stored pkl file
open_file = open("DcpNorthdelhiTweets.pkl", "rb")
tweets = pickle.load(open_file)
open_file.close()

In [ ]:
#users who have been replied by police
replied_users=[]
for i in tweets:
    if(i.in_reply_to_status_id!=None):
        replied_users.append(i.in_reply_to_status_id)  
replied_users=list(set(replied_users))

In [ ]:
#collecting original tweets for the users in replied_users list
original_tweets=[]
c=0
for i in replied_users:
    print(c)
    try:
        original_tweets.append(api.get_status(i,tweet_mode='extended'))
    except:
        original_tweets.append(None)
    c+=1

In [ ]:
#writing tweets in pkl file
# ofile = open("Replied_tweets.pkl", "wb")
# pickle.dump(original_tweets, ofile)
# ofile.close()

In [ ]:
#reading pkl file
ofile = open("Replied_tweets.pkl", "rb")
original_tweets = pickle.load(ofile)
ofile.close()

In [ ]:
original_id_vs_tweet={}
for i in range(len(replied_users)):
    if(original_tweets[i]!=None):
        original_id_vs_tweet[replied_users[i]]=original_tweets[i]

In [ ]:
replied_by_police={}
for i in tweets:
    if(i.in_reply_to_status_id in original_id_vs_tweet):
        replied_by_police[i.in_reply_to_status_id]=i

In [ ]:
#regex
email_regex="(\S(\.|\w)+[@]+(\.|\w)+([.]\S*))"
mobile_number_regex="\D[0-9]{10}\D"
aadhar_number_regex="\d{4}\s\d{4}\s\d{4}"
pincode_regex="\D[1-9]{1}[0-9]{2}[0-9]{3}\D|\D[1-9]{1}[0-9]{2}\D{1}[0-9]{3}\D"
vehicle_number_regex="[A-Z]{2}[0-9]{1,2}(?:[A-Z])?(?:[A-Z]*)?[0-9]{4}" #taken online


In [ ]:
#finding different PII
tweet_id=[]
email_id=[]
mobile_number=[]
aadhar_number=[]
pincode=[]
vehicle_number=[]
email_id_count=[]
mobile_number_count=[]
aadhar_number_count=[]
pincode_count=[]
vehicle_number_count=[]

for i in original_id_vs_tweet.keys():
    data=original_id_vs_tweet[i].full_text
    tweet_id.append(i)
    email_id_count.append(len(re.findall(email_regex,data)))
    mobile_number_count.append(len(re.findall(mobile_number_regex,data)))
    aadhar_number_count.append(len(re.findall(aadhar_number_regex,data)))
    pincode_count.append(len(re.findall(pincode_regex,data)))
    vehicle_number_count.append(len(re.findall(vehicle_number_regex,data)))
    email_id.append(re.findall(email_regex,data))
    mobile_number.append(re.findall(mobile_number_regex,data))
    aadhar_number.append(re.findall(aadhar_number_regex,data))
    pincode.append(re.findall(pincode_regex,data))
    vehicle_number.append(re.findall(vehicle_number_regex,data))

In [ ]:
#PII table 1
data_pii={}
data_pii['PII_types']=['emails','mobile_numbers','aadhar_card_numbers','pincode','vehicle_numbers']
data_pii['count']=[sum(email_id_count),sum(mobile_number_count),sum(aadhar_number_count),sum(pincode_count),sum(vehicle_number_count)]

df_pii_1=pd.DataFrame(data_pii)
df_pii_1


In [ ]:
#PII table 2
data_pii={}
data_pii['Tweet ID']=tweet_id
data_pii['no_of_emails']=email_id_count
data_pii['no_of_mobile_numbers']=mobile_number_count
data_pii['no_of_aadhar_card_no']=aadhar_number_count
data_pii['no_of_pincodes']=pincode_count
data_pii['no_of_vehicle_numbers']=vehicle_number_count

df_pii_1=pd.DataFrame(data_pii)
df_pii_1[(df_pii_1['no_of_emails']>0) | (df_pii_1['no_of_mobile_numbers']>0) | (df_pii_1['no_of_aadhar_card_no']>0) | (df_pii_1['no_of_pincodes']>0) | (df_pii_1['no_of_vehicle_numbers']>0)]

In [ ]:
for i in tweet_id:
    print(original_id_vs_tweet[i].full_text)

In [ ]:
#ques1 part b
media_tweets=[]
media_tweets_id=[]
media_tweets_url=[]
for i in original_id_vs_tweet.keys():
    data=original_id_vs_tweet[i].entities
    if('media' in data):
        media_tweets_id.append(i)
        media_tweets.append(original_id_vs_tweet[i])
        media_tweets_url.append(data['media'][0]['media_url_https'])


In [ ]:
print("Number of tweets with media: ",len(media_tweets))

In [ ]:
#links of images
for i in media_tweets_url:
    print(i)

In [ ]:
#ques2
id_vs_response_time={}

for i in replied_by_police.keys():
    id_vs_response_time[i]=replied_by_police[i].created_at-original_id_vs_tweet[i].created_at

In [ ]:
#finding response time for tweets
response_times=id_vs_response_time.values()
response_times_seconds=[i.total_seconds() for i in response_times]

In [ ]:
#finding asked values
mean_response_time=datetime.timedelta(seconds=st.mean(response_times_seconds))
minimum_response_time=min(response_times)
maximum_response_time=max(response_times)
standard_deviation_of_response_time=datetime.timedelta(seconds=st.stdev(response_times_seconds))

In [ ]:
print("mean_response_time:",mean_response_time)
print("minimum_response_time:",minimum_response_time)
print("maximum_response_time:",maximum_response_time)
print("standard_deviation_of_response_time:",standard_deviation_of_response_time)

In [ ]:
x_axis=[]
y_axis=[]

for i in replied_by_police.keys():
    x_axis.append(replied_by_police[i].created_at)
    y_axis.append(id_vs_response_time[i].total_seconds())

dc={}
dc["Replied_time_of_police"]=x_axis
dc["Response_time"]=y_axis

df_dc=pd.DataFrame(dc)
df_dc=df_dc.sort_values(by='Replied_time_of_police')

grf=plotly.express.line(df_dc,x='Replied_time_of_police',y='Response_time' ,title="Replied tweets of police vs their response time")
grf.show()

In [ ]:
#ques3
#1. appreciation 2. report 3. requests 4. accountability 5. other
table_id=[]
table_tweet=[]
table_classified=[]

for i in list(original_id_vs_tweet.values())[70:100]:
    print(i.full_text)
    j=input()
    table_id.append(i.id)
    table_tweet.append(i.full_text)
    table_classified.append(j)

In [ ]:
dc_q3={}
dc_q3['Tweet_id']=table_id
dc_q3['Tweet']=table_tweet
dc_q3['classification']=table_classified

dc_q3=pd.DataFrame(dc_q3)
dc_q3